In [2]:
import xarray as xr
import glob
import numpy as np

# Directory containing the NetCDF files
data_dir = "C:/Users/31610/Documents/UvA/Master thesis/Data/ERA5/"
file_pattern = f"{data_dir}*.nc"

# Step 1: Find and sort all NetCDF files
file_list = sorted(glob.glob(file_pattern))  # Sort to ensure chronological order
print(f"Found {len(file_list)} files: {file_list}")

# Define NAO region coordinates
lat_min, lat_max = 20, 80  # 20°N to 80°N
lon_min_360, lon_max_360 = 270, 40  # 90°W (270°E) to 40°E

# Step 2: Process each file
processed_datasets = []
for file in file_list:
    # Open the dataset
    ds = xr.open_dataset(file, decode_times=False)
    ds = ds.isel(latitude=slice(None, None, 4), longitude=slice(None, None, 4))

    # Select the two parts of the longitude range
    lon_part1 = ds.sel(
        latitude=slice(lat_max, lat_min),  # 80°N to 20°N
        longitude=slice(lon_min_360, 360)  # 270°E to 360°
    )

    lon_part2 = ds.sel(
        latitude=slice(lat_max, lat_min),  # 80°N to 20°N
        longitude=slice(0, lon_max_360)  # 0° to 40°E
    )

    # Concatenate the two longitude parts
    nao_data = xr.concat([lon_part1, lon_part2], dim='longitude')

    # Decode the time coordinate
    nao_data = xr.decode_cf(nao_data)

    # Append to list of processed datasets
    processed_datasets.append(nao_data)


Found 2 files: ['C:/Users/31610/Documents/UvA/Master thesis/Data/ERA5\\1940-1980.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/ERA5\\1981-2025.nc']


In [3]:
merged_nao_data = xr.concat(processed_datasets, dim='valid_time', data_vars='all', coords='different')

In [4]:
# Sort by time to ensure chronological order
merged_nao_data = merged_nao_data.sortby('valid_time')

# Step 4: Verify the merged dataset
print("Merged dataset shape:", merged_nao_data.dims)
print("Time range:", merged_nao_data['valid_time'].min().values, "to", merged_nao_data['valid_time'].max().values)
print("MSL shape:", merged_nao_data['msl'].shape)

# Step 5: Save the merged dataset
output_file = "C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/merged_nao_data_1940-2025.nc"
merged_nao_data.to_netcdf(output_file)
print(f"Saved merged dataset to {output_file}")
# Step 6: Clean up
for ds in processed_datasets:
    ds.close()  # Close each dataset to free up resources

Merged dataset shape: FrozenMappingWarningOnValuesAccess({'valid_time': 12947, 'latitude': 61, 'longitude': 131})
Time range: 1940-01-01T00:00:00.000000000 to 2025-03-31T00:00:00.000000000
MSL shape: (12947, 61, 131)
Saved merged dataset to C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/merged_nao_data_1940-2025.nc
